In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [22]:
df_weekly = pd.read_csv('../data/clean/df_weekly.csv')

In [6]:
# Daten normalisieren
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_weekly[['confirmed', 'deaths' ,"recovered"
,"vaccines","people_vaccinated","people_fully_vaccinated"
,"school_closing","workplace_closing","cancel_events"
,"gatherings_restrictions","transport_closing"
,"stay_home_restrictions","internal_movement_restrictions"
,"internal_movement_restrictions","information_campaigns","testing_policy"
,"contact_tracing","facial_coverings"
,"vaccination_policy","elderly_people_protection","population"]])
new_data = np.array([[39919.42857142857, 1132.2857142857142, 38787.142857142855, 8832.0, 8832.0, 0.0, 3.0, 3.0, 2.0, 4.0, 1.0, 1.4285714285714286, -1.5714285714285714,
                      3.0, 2.0, 1.0, 1.1428571428571428, 2.0, 2.0, 2.0, 1841179.0]])  # Beispielhafte Eingangsdaten (Features)
# Vorbereitung der Eingangsdaten (Features) und der Zielvariablen
features = scaled_data[:, :-1]
target = scaled_data[:, -1]


train_size = int(len(df_weekly) * 0.8)
test_size = len(df_weekly) - train_size
train_features, test_features = features[:train_size], features[train_size:]
train_target, test_target = target[:train_size], target[train_size:]

# feature -> target
# t1 -> t2
def create_lstm_dataset(features, target, time_steps):
    X, y = [], []
    for i in range(len(features) - time_steps):
        X.append(features[i:(i+time_steps)])
        y.append(target[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 5 # Das problem mit der input Größe >.<. Aktuell werden alle 5 daten aus den train data aneinandergereiht. Hier muss noch eine lösung her
train_features_lstm, train_target_lstm = create_lstm_dataset(train_features, train_target, time_steps)
test_features_lstm, test_target_lstm = create_lstm_dataset(test_features, test_target, time_steps)


model = Sequential()
model.add(LSTM(units=64, input_shape=(time_steps, train_features_lstm.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse', run_eagerly=True)


history = model.fit(train_features_lstm, train_target_lstm, epochs=3, batch_size=32, validation_data=(test_features_lstm, test_target_lstm))


loss = model.evaluate(test_features_lstm, test_target_lstm)

print('Test Loss:', loss)


Epoch 1/3
1020/1020 [==============================] - 36s 36ms/step - loss: 0.0016 - val_loss: 0.0055
Epoch 2/3
1020/1020 [==============================] - 36s 35ms/step - loss: 0.0010 - val_loss: 0.0041
Epoch 3/3
255/255 [==============================] - 4s 16ms/step - loss: 0.0044
Test Loss: 0.004425948951393366
